# Instrument Pricing Analytics - Floating Rate Notes

#### Refinitiv Workspace Calculators
**Floating Rate Note Calculator**

![Floating Rate Note Calc](FRNCalcPriceYieldDM2021.png)

As with the Fixed Rate Bond calculator, the Floating Rate Note Calculator also uses the Instrument Pricing Analytics APIs behind the scenes for its computations.

#### Replicate Desktop Calculations with Python code


#### Initialisation


In [1]:
import refinitiv.dataplatform as rdp
from refinitiv.dataplatform.content import ipa
from refinitiv.dataplatform.content.ipa import bond
import json
import configparser as cp
from IPython.display import HTML
import pandas as pd

RDP Library is currently available in 
* Python, 
* .NET versions
* TypeScript (soon)

RDP APIs are implemented as RESTful APIs -  use other languages to HTTP POST the request &  parse the Response.  

Using RDP libraries? Saves some overhead - authentication, building Request message, parsing Response etc.

#### Connect to the Refintiv Data Platform

In [2]:
config = cp.ConfigParser()
config.read("config.cfg")

['config.cfg']

In [3]:
#rdp.open_desktop_session(config['desktop']['app_key'])

#### OR

In [4]:
session = rdp.open_platform_session(
    config['session']['app_key'],
    rdp.GrantPassword( username=config['session']['user'], password=config['session']['password'] )
)

### Example 1 : Get Price, Yield and Discount Margin

Function Layer example

In [5]:
rdp.get_bond_analytics(
    universe = [
        "XS2104969550",
        "22532XPE2=",
        "CA210239502=",
        "SGXF41098645",
        "CA298785HY55",
        "XS0122667230"
    ],
    calculation_params = bond.CalculationParams(
        market_data_date="2021-03-03",
        price_side = ipa.enum_types.PriceSide.BID
    ),
    fields = ["ISIN","InstrumentDescription","RedemptionDate","CouponFormula","IndexRic",
              "CleanPrice", "DiscountMarginBp", "NeutralYieldPercent", 
              "CurrentIndexPercent","ProjectedIndexPercent"
             ]
)

,ISIN,InstrumentDescription,RedemptionDate,CouponFormula,IndexRic,CleanPrice,DiscountMarginBp,NeutralYieldPercent,CurrentIndexPercent,ProjectedIndexPercent
0,None,EIB 1.437 01/22/23 FRN,2023-01-22T00:00:00Z,"ROUND(CMP(ESTR), 0.0001) + 2",EUROSTR=,103.804,1.98485,-0.545152,-0.564,-0.565
1,None,CAGR 0.410 11/15/21 FRN,2021-11-15T00:00:00Z,SOFR + 0.40,USDSOFR=,100.1944,12.796708,0.167967,0.02,0.04
2,None,EDC 0.379 01/17/25 FRN MTN,2025-01-17T00:00:00Z,"ROUND(CMP(SONIA), 0.00001) + 0.33",SONIAOSR=,100.418,22.153907,0.270639,0.0491,0.0491
3,None,DBSM 0.873 05/14/21 FRN,2021-05-14T00:00:00Z,"ROUND(CMP(SORA), 0.0001) + 0.65",SORA=MAST,100.128,-0.230241,0.064998,0.2477,0.0673
4,None,EIB 0.421 05/24/22 FRN,2022-05-24T00:00:00Z,"ROUND(CMP(CORRA), 0.00001) + 0.25",CORRA=,99.895,33.641217,0.496412,0.2,0.16
5,None,HSHND 01/22/41 FRN MTN,2041-01-22T00:00:00Z,EIEUR3M + 0.38,EURIBOR3MD=,68.166,245.04834,1.904483,-0.543,-0.546



For the remaining examples, I will use the Endpoint interface


In [6]:
financial_contract_api = rdp.Endpoint(session, 
            'https://api.refinitiv.com/data/quantitative-analytics/v1/financial-contracts')

### Example 2: Compute Cash Flows
Another use of the same API is to Compute Cash flows e.g.:

In [7]:
request_body={
    "fields": [
        "CashFlowDatesArray",
        "CashFlowAnnualRatesArray",
        "CashFlowInterestPercentsArray",
        "CashFlowInterestAmountsInDealCcyArray",
        "CashFlowCapitalAmountsInDealCcyArray"
        #"CashFlowTotalPercentsArray",
        #"CashFlowTotalAmountsInDealCcyArray",
        #"CashFlowNormalizationFactorsArray",
        #"CashFlowResidualAmountsInDealCcyArray",
        #"CashFlowCapitalPercentsArray",
    ],
    "outputs": [
        "Data", "Headers"
    ],
    "universe": [
        {
            "instrumentType": "Bond",
            "instrumentDefinition": {
                "instrumentCode": "22532XPE2="
            },
            "pricingParameters": {
                "marketDataDate": "2021-03-03",
                "priceSide": "Bid",
                "computeCashFlowFromIssueDate":"true"
            }
        }
    ]
}

response = financial_contract_api.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body
)
df = pd.DataFrame(data=[item for elem in response.data.raw['data'] for item in elem]).transpose()
df.columns = [h['name'] for h in response.data.raw['headers']]
df

,CashFlowDatesArray,CashFlowAnnualRatesArray,CashFlowInterestPercentsArray,CashFlowInterestAmountsInDealCcyArray,CashFlowCapitalAmountsInDealCcyArray
0,2020-05-15T00:00:00Z,0.926,0.234111,2341.111111,0.0
1,2020-08-17T00:00:00Z,0.483,0.126117,1261.166667,0.0
2,2020-11-16T00:00:00Z,0.487,0.123103,1231.027778,0.0
3,2021-02-16T00:00:00Z,0.472,0.120622,1206.222222,0.0
4,2021-05-17T00:00:00Z,0.438,0.1095,1095.0,0.0
5,2021-08-16T00:00:00Z,0.44,0.111222,1112.222222,0.0
6,2021-11-15T00:00:00Z,0.44,0.111222,1112.222222,1000000.0


### Example 3: Price from Discount Margin assumptions
We can also compute the price for different discount margin values:


In [8]:
request_body={
    "fields": [
              "InstrumentTag","CleanPrice", "DiscountMarginBp","MarketValueInReportCcy",
            "ModifiedDuration", "SpreadDuration", "Convexity", "AverageLife", "DV01Bp"
    ],
   "outputs": [
        "Data",
        "Headers"
    ],
   "universe": [
        {
            "instrumentType": "Bond",
            "instrumentDefinition": {
                "InstrumentTag" : "computed margin",
                "instrumentCode": "XS2104969550"
            },
            "pricingParameters": {
                "marketDataDate": "2021-02-25"
            }
        }, {
            "instrumentType": "Bond",
            "instrumentDefinition": {
                "InstrumentTag" : "Margin of 5bp",
                "instrumentCode": "XS2104969550"
            },
            "pricingParameters": {
                "marketDataDate": "2021-02-25",
                "discountMarginBp":5
            }
        },
        {
            "instrumentType": "Bond",
            "instrumentDefinition": {
                 "InstrumentTag" : "Margin of 10 bp",
                "instrumentCode": "XS2104969550"
            },
            "pricingParameters": {
                "marketDataDate": "2021-02-25",
                "discountMarginBp":10
            }
        }
    ]
}

response = financial_contract_api.send_request(
    method = rdp.Endpoint.RequestMethod.POST,
    body_parameters = request_body
)
pd.DataFrame(data=response.data.raw['data'], columns=[h['name'] for h in response.data.raw['headers']])

,InstrumentTag,CleanPrice,DiscountMarginBp,MarketValueInReportCcy,ModifiedDuration,SpreadDuration,Convexity,AverageLife,DV01Bp
0,computed margin,103.8385,1.355409,1.039902e+06,0.144444,1.897630,4.015869,1.897222,1.949965
1,Margin of 5bp,103.7660,5.000000,1.039177e+06,0.144444,1.897447,4.015091,1.897222,1.948412
2,Margin of 10 bp,103.6680,10.000000,1.038197e+06,0.144444,1.897200,4.014040,1.897222,1.946314
